In [2]:
import pandas as pd
import ydata_profiling
import requests
import sqlite3

In [3]:
df = pd.read_csv('e_goods_sales.csv')
df.head(5)

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,176558,USB-C Charging Cable,2,11.95,04/19/19 08:46,"917 1st St, Dallas, TX 75001"
1,NaN,NaN,NaN,NaN,NaN,NaN
2,176559,Bose SoundSport Headphones,1,99.99,04/07/19 22:30,"682 Chestnut St, Boston, MA 02215"
3,176560,Google Phone,1,600,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"
4,176560,Wired Headphones,1,11.99,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"


In [5]:
df.shape

(186850, 6)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186850 entries, 0 to 186849
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Order ID          186305 non-null  object
 1   Product           186305 non-null  object
 2   Quantity Ordered  186305 non-null  object
 3   Price Each        186305 non-null  object
 4   Order Date        186305 non-null  object
 5   Purchase Address  186305 non-null  object
dtypes: object(6)
memory usage: 8.6+ MB


In [8]:
#делаем профайлинг отчет датафрейма
df.profile_report()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [9]:
df.columns

Index(['Order ID', 'Product', 'Quantity Ordered', 'Price Each', 'Order Date',
       'Purchase Address'],
      dtype='object')

In [11]:
#переименовываем колонки
df.rename(columns=
{
          'Order ID' : 'order_id',
          'Product' : 'product',
          'Quantity Ordered' : 'quantity_ordered',
          'Price Each' : 'price',
          'Order Date' : 'order_date', 
          'Purchase Address' : 'purchase_address'
},
inplace=True)

In [12]:
df.columns

Index(['order_id', 'product', 'quantity_ordered', 'price', 'order_date',
       'purchase_address'],
      dtype='object')

In [13]:
df.isnull().sum()

order_id            545
product             545
quantity_ordered    545
price               545
order_date          545
purchase_address    545
dtype: int64

In [17]:
#смотрим пустые значения в данных
df[df.isna().any(axis=1)]

,order_id,product,quantity_ordered,price,order_date,purchase_address
1,NaN,NaN,NaN,NaN,NaN,NaN
356,NaN,NaN,NaN,NaN,NaN,NaN
735,NaN,NaN,NaN,NaN,NaN,NaN
1433,NaN,NaN,NaN,NaN,NaN,NaN
1553,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
185176,NaN,NaN,NaN,NaN,NaN,NaN
185438,NaN,NaN,NaN,NaN,NaN,NaN
186042,NaN,NaN,NaN,NaN,NaN,NaN
186548,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
#убираем пустые значения
df = df[~df.isna().any(axis=1)]

In [20]:
df.shape

(186305, 6)

In [22]:
#проверяем данные на дубликаты
df[df.duplicated()].sort_values('order_id')

,order_id,product,quantity_ordered,price,order_date,purchase_address
68422,142071,AA Batteries (4-pack),1,3.84,01/17/19 23:02,"131 2nd St, Boston, MA 02215"
71673,145143,Lightning Charging Cable,1,14.95,01/06/19 03:01,"182 Jefferson St, San Francisco, CA 94016"
73358,146765,Google Phone,1,600,01/21/19 11:23,"918 Highland St, New York City, NY 10001"
74354,147707,Wired Headphones,1,11.99,01/04/19 16:50,"883 4th St, Dallas, TX 75001"
75681,148984,USB-C Charging Cable,1,11.95,01/08/19 17:36,"562 14th St, Boston, MA 02215"
...,...,...,...,...,...,...
30648,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
100559,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
100547,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
15280,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address


In [24]:
df[df['order_id'] == '145143']

,order_id,product,quantity_ordered,price,order_date,purchase_address
71672,145143,Lightning Charging Cable,1,14.95,01/06/19 03:01,"182 Jefferson St, San Francisco, CA 94016"
71673,145143,Lightning Charging Cable,1,14.95,01/06/19 03:01,"182 Jefferson St, San Francisco, CA 94016"


In [25]:
#удаление дубликатов
df = df.drop_duplicates()

In [26]:
df.shape

(185687, 6)

In [28]:
df[df.duplicated()].sort_values('order_id')

,order_id,product,quantity_ordered,price,order_date,purchase_address


In [29]:
len(df[df['order_id'].duplicated()])

7249

In [30]:
df[df['order_id'].duplicated()]

,order_id,product,quantity_ordered,price,order_date,purchase_address
4,176560,Wired Headphones,1,11.99,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"
19,176574,USB-C Charging Cable,1,11.95,04/03/19 19:42,"20 Hill St, Los Angeles, CA 90001"
33,176586,Google Phone,1,600,04/10/19 17:00,"365 Center St, San Francisco, CA 94016"
120,176672,USB-C Charging Cable,1,11.95,04/12/19 11:07,"778 Maple St, New York City, NY 10001"
130,176681,ThinkPad Laptop,1,999.99,04/20/19 10:39,"331 Cherry St, Seattle, WA 98101"
...,...,...,...,...,...,...
186762,259277,Wired Headphones,2,11.99,09/28/19 13:07,"795 Willow St, New York City, NY 10001"
186784,259297,Lightning Charging Cable,1,14.95,09/15/19 18:54,"138 Main St, Boston, MA 02215"
186792,259303,AA Batteries (4-pack),1,3.84,09/20/19 20:18,"106 7th St, Atlanta, GA 30301"
186804,259314,AAA Batteries (4-pack),2,2.99,09/16/19 00:25,"241 Highland St, Atlanta, GA 30301"


In [31]:
#видно, что order_id может дублироваться при покупке разных товаров. возможно в одной покупке приобретено несколько товаров
df[df['order_id'] == '176560']

,order_id,product,quantity_ordered,price,order_date,purchase_address
3,176560,Google Phone,1,600,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"
4,176560,Wired Headphones,1,11.99,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"


In [32]:
df.nunique()

order_id            178438
product                 20
quantity_ordered        10
price                   24
order_date          142396
purchase_address    140788
dtype: int64

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 185687 entries, 0 to 186849
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   order_id          185687 non-null  object
 1   product           185687 non-null  object
 2   quantity_ordered  185687 non-null  object
 3   price             185687 non-null  object
 4   order_date        185687 non-null  object
 5   purchase_address  185687 non-null  object
dtypes: object(6)
memory usage: 9.9+ MB


In [35]:
#меняем типы данных
df.price = df.price.astype('float')

ValueError: could not convert string to float: 'Price Each'

In [36]:
df[df.price == 'Price Each']

,order_id,product,quantity_ordered,price,order_date,purchase_address
519,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address


In [37]:
df.drop(519, inplace=True)

In [38]:
df[df.price == 'Price Each']

,order_id,product,quantity_ordered,price,order_date,purchase_address


In [39]:
#меняем типы данных
df.price = df.price.astype('float')

In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 185686 entries, 0 to 186849
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   order_id          185686 non-null  object 
 1   product           185686 non-null  object 
 2   quantity_ordered  185686 non-null  object 
 3   price             185686 non-null  float64
 4   order_date        185686 non-null  object 
 5   purchase_address  185686 non-null  object 
dtypes: float64(1), object(5)
memory usage: 9.9+ MB


In [42]:
#меняем тип данных со строки на дату
df.order_date = df.order_date.apply(pd.to_datetime)

In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 185686 entries, 0 to 186849
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   order_id          185686 non-null  object        
 1   product           185686 non-null  object        
 2   quantity_ordered  185686 non-null  object        
 3   price             185686 non-null  float64       
 4   order_date        185686 non-null  datetime64[ns]
 5   purchase_address  185686 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 9.9+ MB


In [45]:
df.head()

,order_id,product,quantity_ordered,price,order_date,purchase_address
0,176558,USB-C Charging Cable,2,11.95,2019-04-19 08:46:00,"917 1st St, Dallas, TX 75001"
2,176559,Bose SoundSport Headphones,1,99.99,2019-04-07 22:30:00,"682 Chestnut St, Boston, MA 02215"
3,176560,Google Phone,1,600.00,2019-04-12 14:38:00,"669 Spruce St, Los Angeles, CA 90001"
4,176560,Wired Headphones,1,11.99,2019-04-12 14:38:00,"669 Spruce St, Los Angeles, CA 90001"
5,176561,Wired Headphones,1,11.99,2019-04-30 09:27:00,"333 8th St, Los Angeles, CA 90001"


In [50]:
df['price'].describe()

count    185686.000000
mean        184.519255
std         332.843838
min           2.990000
25%          11.950000
50%          14.950000
75%         150.000000
max        1700.000000
Name: price, dtype: float64

In [51]:
#меняем тип данных 
df['quantity_ordered'] = df['quantity_ordered'].astype('int')

In [52]:
#вычисляем сумму заказа по строкам
df['order_amount'] = df['price'] * df['quantity_ordered']

In [53]:
#смотрим статистику по числовым столбцам
df[['quantity_ordered', 'price', 'order_amount']].describe()

,quantity_ordered,price,order_amount
count,185686.000000,185686.000000,185686.000000
mean,1.124544,184.519255,185.611936
std,0.443069,332.843838,333.032118
min,1.000000,2.990000,2.990000
25%,1.000000,11.950000,11.950000
50%,1.000000,14.950000,14.950000
75%,1.000000,150.000000,150.000000
max,9.000000,1700.000000,3400.000000


In [54]:
import datetime

In [56]:
df['order_date'] = df['order_date'].dt.date

In [57]:
df.head()

,order_id,product,quantity_ordered,price,order_date,purchase_address,order_amount
0,176558,USB-C Charging Cable,2,11.95,2019-04-19,"917 1st St, Dallas, TX 75001",23.90
2,176559,Bose SoundSport Headphones,1,99.99,2019-04-07,"682 Chestnut St, Boston, MA 02215",99.99
3,176560,Google Phone,1,600.00,2019-04-12,"669 Spruce St, Los Angeles, CA 90001",600.00
4,176560,Wired Headphones,1,11.99,2019-04-12,"669 Spruce St, Los Angeles, CA 90001",11.99
5,176561,Wired Headphones,1,11.99,2019-04-30,"333 8th St, Los Angeles, CA 90001",11.99


In [59]:
df['product'].unique()

array(['USB-C Charging Cable', 'Bose SoundSport Headphones',
       'Google Phone', 'Wired Headphones', 'Macbook Pro Laptop',
       'Lightning Charging Cable', '27in 4K Gaming Monitor',
       'AA Batteries (4-pack)', 'Apple Airpods Headphones',
       'AAA Batteries (4-pack)', 'iPhone', 'Flatscreen TV',
       '27in FHD Monitor', '20in Monitor', 'LG Dryer', 'ThinkPad Laptop',
       'Vareebadd Phone', 'LG Washing Machine', '34in Ultrawide Monitor'],
      dtype=object)

In [60]:
#считаем сумму заказов по каждому продукту
df.groupby('product')['order_amount'].sum().sort_values(ascending=False)

product
Macbook Pro Laptop            8032500.00
iPhone                        4792900.00
ThinkPad Laptop               4127958.72
Google Phone                  3317400.00
27in 4K Gaming Monitor        2433147.61
34in Ultrawide Monitor        2352898.08
Apple Airpods Headphones      2345550.00
Flatscreen TV                 1443900.00
Bose SoundSport Headphones    1342865.70
27in FHD Monitor              1131074.59
Vareebadd Phone                827200.00
20in Monitor                   453818.74
LG Washing Machine             399600.00
LG Dryer                       387600.00
Lightning Charging Cable       346376.55
USB-C Charging Cable           285975.45
Wired Headphones               246082.76
AA Batteries (4-pack)          106041.60
AAA Batteries (4-pack)          92648.14
Name: order_amount, dtype: float64

In [61]:
#подключаемся и создаем новую базу SQLite
con = sqlite3.connect('db_good_sales')

In [62]:
df.to_sql(name='e_sales_test',con=con, index=False)

185686